# Genetic Analysis Autoencoder and baseline modeling

## Install and import libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import sys
# add the path of the virtual environmentsite-packages to colab system path
sys.path.append("/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages")


In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate;

In [ ]:
from pandas_plink import read_plink1_bin # Not pandas, lets you read in plink files as xarray::dataArray
import pandas as pd
import numpy as np
import xarray as xr
#import matplotlib.pyplot as plt
import tensorflow as tf
#from sklearn.metrics import accuracy_score, precision_score, recall_score
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.random import sample_without_replacement

## Read in ADNI Dataset

In [ ]:
fPATH_ADNI_Omni25M = '/content/drive/MyDrive/ADV_DS/ADNI_GENETIC_DATA/ADNI_Omni25M/microarray_SNP_data/WGS_Omni25_BIN.bed'
fPATH_INFO = '/content/drive/MyDrive/ADV_DS/ADNI_GENETIC_DATA/ADNI_Omni25M/ADNI_Deletions_Radke/ADNI_IDinfo.txt' # this file contains our phenotypes which we will need later.

In [ ]:
DF_idInfo = pd.read_csv(fPATH_INFO, delimiter='\t', usecols=['ADNI_PTID', 'pheno'])

In [ ]:
DF_idInfo

,ADNI_PTID,pheno
0,032_S_0978,3
1,021_S_2125,2
2,036_S_0869,3
3,098_S_4215,3
4,099_S_4104,1
...,...,...
803,041_S_1260,2
804,024_S_4280,3
805,037_S_0501,2
806,057_S_0934,1


In [ ]:
dArray_ADNI_Omni25M = read_plink1_bin(fPATH_ADNI_Omni25M)

Mapping files: 100%|██████████| 3/3 [00:11<00:00,  3.74s/it]


In [ ]:
dArray_ADNI_Omni25M

<xarray.DataArray 'genotype' (sample: 812, variant: 2379855)>
dask.array<transpose, shape=(812, 2379855), dtype=float32, chunksize=(812, 1024), chunktype=numpy.ndarray>
Coordinates: (12/14)
  * sample   (sample) object '067_S_0056' '123_S_0108' ... '135_S_4356'
  * variant  (variant) <U14 'variant0' 'variant1' ... 'variant2379854'
    fid      (sample) object '1' '2' '3' '4' '5' ... '815' '816' '817' '818'
    iid      (sample) object '067_S_0056' '123_S_0108' ... '135_S_4356'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    ...       ...
    chrom    (variant) object '0' '0' '0' '0' '0' ... '26' '26' '26' '26' '26'
    snp      (variant) object 'rs2698846' 'rs2542903' ... '200610-37'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 ... 100.0 100.0 100.0 100.0
    pos      (variant) int32 0 0 0 0 0 0 ... 16329 16358 16364 16393 16401 16484
    a0       (variant) object '0' '0' '0' 'T' 'A' '0' ... 'C' 'C' 'A' 'G' 'G'
    a1       (variant) object 'G' 'T' 'T' 'A' 'T' 'T' ... 'T' 'T' 'G' 'A' 'A'

## Data Cleaning

### Data loads in on wrong axis and needs flipped

In [ ]:
dArray_ADNI_Omni25M = dArray_ADNI_Omni25M.T
dArray_ADNI_Omni25M

<xarray.DataArray 'genotype' (variant: 2379855, sample: 812)>
dask.array<transpose, shape=(2379855, 812), dtype=float32, chunksize=(1024, 812), chunktype=numpy.ndarray>
Coordinates: (12/14)
  * sample   (sample) object '067_S_0056' '123_S_0108' ... '135_S_4356'
  * variant  (variant) <U14 'variant0' 'variant1' ... 'variant2379854'
    fid      (sample) object '1' '2' '3' '4' '5' ... '815' '816' '817' '818'
    iid      (sample) object '067_S_0056' '123_S_0108' ... '135_S_4356'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    ...       ...
    chrom    (variant) object '0' '0' '0' '0' '0' ... '26' '26' '26' '26' '26'
    snp      (variant) object 'rs2698846' 'rs2542903' ... '200610-37'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 ... 100.0 100.0 100.0 100.0
    pos      (variant) int32 0 0 0 0 0 0 ... 16329 16358 16364 16393 16401 16484
    a0       (variant) object '0' '0' '0' 'T' 'A' '0' ... 'C' 'C' 'A' 'G' 'G'
    a1       (variant) object 'G' 'T' 'T' 'A' 'T' 'T' ... 'T' 'T' 'G' 'A' 'A'

### Drop columns without values

In [ ]:
# drop empty columns
dArray_Omni25= dArray_ADNI_Omni25M.drop_vars(['mother', 'father'])


### find missinng Patient_IDs and drop them from our dataset.

- Because the ADNI_idInfo file has four less patients than the genome file, and because the ADNI_Info file contains the phenotypes of our patients, we will drop the missing patients from the genome file.

In [ ]:
# find missing patients
iid_arr = dArray_Omni25.sample['iid'].values

setOf_dFrame_ptids = set(DF_idInfo['ADNI_PTID'])
setOf_dArray_ptids = set(iid_arr)
missing_ids = setOf_dArray_ptids - setOf_dFrame_ptids
missing_ids = np.array(list(missing_ids))
print(f'Missing Patient Ids:', missing_ids)

Missing Patient Ids: ['035_S_0292' '023_S_0376' '012_S_1212' '137_S_4536']


In [ ]:
#drop missing patients from d_array
drop_set = set(missing_ids)
mask = np.array([iid not in drop_set for iid in dArray_Omni25.sample['iid'].values])
resized_dArray = dArray_Omni25.sel(sample=mask)

## Update the 'trait' column in the data array with the phenotypes 'pheno' column from the ADNI_idInfo file

In [ ]:
# sort dataArray by patient id:
sorted_dArray = resized_dArray.sortby('iid')
print(sorted_dArray)
# sort id_info Dataframe
sorted_df = DF_idInfo.sort_values(by='ADNI_PTID')
print(sorted_df)
# replace the empty trait values with the new pheno values from the idInfo dataframe
sorted_dArray.sample.trait.values = sorted_df['pheno'].to_numpy()
dArray_Omni25M = sorted_dArray

<xarray.DataArray 'genotype' (variant: 2379855, sample: 808)>
dask.array<getitem, shape=(2379855, 808), dtype=float32, chunksize=(1024, 808), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object '002_S_0413' '002_S_0685' ... '941_S_4420'
  * variant  (variant) <U14 'variant0' 'variant1' ... 'variant2379854'
    fid      (sample) object '199' '702' '57' '158' ... '389' '248' '253' '534'
    iid      (sample) object '002_S_0413' '002_S_0685' ... '941_S_4420'
    gender   (sample) object '2' '2' '2' '1' '2' '1' ... '1' '1' '1' '2' '2' '1'
    trait    (sample) object '-9' '-9' '-9' '-9' '-9' ... '-9' '-9' '-9' '-9'
    chrom    (variant) object '0' '0' '0' '0' '0' ... '26' '26' '26' '26' '26'
    snp      (variant) object 'rs2698846' 'rs2542903' ... '200610-37'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 ... 100.0 100.0 100.0 100.0
    pos      (variant) int32 0 0 0 0 0 0 ... 16329 16358 16364 16393 16401 16484
    a0       (variant) object '0' '0' '0' 'T' 'A' '0' ...

In [ ]:
dArray_Omni25M

<xarray.DataArray 'genotype' (variant: 2379855, sample: 808)>
dask.array<getitem, shape=(2379855, 808), dtype=float32, chunksize=(1024, 808), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object '002_S_0413' '002_S_0685' ... '941_S_4420'
  * variant  (variant) <U14 'variant0' 'variant1' ... 'variant2379854'
    fid      (sample) object '199' '702' '57' '158' ... '389' '248' '253' '534'
    iid      (sample) object '002_S_0413' '002_S_0685' ... '941_S_4420'
    gender   (sample) object '2' '2' '2' '1' '2' '1' ... '1' '1' '1' '2' '2' '1'
    trait    (sample) int64 1 1 3 2 1 3 1 1 2 2 3 1 ... 2 2 2 2 1 2 1 1 1 1 2 2
    chrom    (variant) object '0' '0' '0' '0' '0' ... '26' '26' '26' '26' '26'
    snp      (variant) object 'rs2698846' 'rs2542903' ... '200610-37'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 ... 100.0 100.0 100.0 100.0
    pos      (variant) int32 0 0 0 0 0 0 ... 16329 16358 16364 16393 16401 16484
    a0       (variant) object '0' '0' '0' 'T' 'A' '0' ... 'C' 'C' 'A' 'G' 'G'
    a1       (variant) object 'G' 'T' 'T' 'A' 'T' 'T' ... 'T' 'T' 'G' 'A' 'A'

## Take Random Sample for processing and analysis

In [ ]:
def get_srs_indices(data: np.array, scaler: int):
  # data is a dim(2, n) array
  # data[0] is your trait you are scaling your sample for
  # data[1] array is your population
  # scaler is your sample size.

  # get phenotype and their counts
  traits, tsize = np.unique(data[0], return_counts = True)
  print('Population:')
  print(f'phenotypes: {traits}, counts: {tsize}')

  # calculate frequencies of phenotypes
  tprops = []
  total = data[0].size
  tprops = [tsize[i]/total for i in range(tsize.size)]

  # get random, scaled samples for each trait as indices
  num_tsamples = [(scaler*tprops[i]) for i in range(tsize.size)]
  p_indexes = [sample_without_replacement(total, num_tsamples[i]) for i in range(traits.size)]
  iids = data[1]
  temp_id_arrays = []

  # get individual ids according to there index from population
  for arr in p_indexes:
    temp = [iids[j] for j in arr]
    temp_id_arrays.append(np.array(temp))

  sample_id_array = np.concatenate(temp_id_arrays)
  # return the repective arrays
  return sample_id_array

In [ ]:
def scaled_random_sample(data: xr.DataArray, sample_size: int):
  d_traits = data.trait.values
  d_iids = data.iid.values
  sample = get_srs_indices([d_traits, d_iids], sample_size)
  mask = np.isin(d_iids, sample)
  selected = data.sel(sample=mask)
  return(selected)

In [ ]:
sample_size = 33
dArray_sample = scaled_random_sample(dArray_Omni25M, sample_size)
dArray_sample

Population:
phenotypes: [1 2 3], counts: [254 369 185]


<xarray.DataArray 'genotype' (variant: 2379855, sample: 32)>
dask.array<getitem, shape=(2379855, 32), dtype=float32, chunksize=(1024, 32), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object '007_S_2058' '007_S_4620' ... '137_S_4672'
  * variant  (variant) <U14 'variant0' 'variant1' ... 'variant2379854'
    fid      (sample) object '82' '559' '17' '260' ... '503' '592' '568' '39'
    iid      (sample) object '007_S_2058' '007_S_4620' ... '137_S_4672'
    gender   (sample) object '2' '1' '1' '1' '1' '1' ... '1' '1' '1' '2' '2' '1'
    trait    (sample) int64 2 1 1 2 3 1 2 1 1 3 2 1 ... 3 3 1 2 1 3 2 3 3 2 1 3
    chrom    (variant) object '0' '0' '0' '0' '0' ... '26' '26' '26' '26' '26'
    snp      (variant) object 'rs2698846' 'rs2542903' ... '200610-37'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 ... 100.0 100.0 100.0 100.0
    pos      (variant) int32 0 0 0 0 0 0 ... 16329 16358 16364 16393 16401 16484
    a0       (variant) object '0' '0' '0' 'T' 'A' '0' ... 'C' 'C' 'A' 'G' 'G'
    a1       (variant) object 'G' 'T' 'T' 'A' 'T' 'T' ... 'T' 'T' 'G' 'A' 'A'

## Expected Genotypes:
1. 0 Homozygous for first allele in bim file:
2. 1 Missing genotype
3. 2 Heterozygous
4. 3 Homozygous for second allele in the bim file.

In [ ]:
vals, counts = np.unique(dArray_sample.values, return_counts=True)

In [ ]:
for i in range(vals.size):
  print(f'{vals[i]}: {counts[i]}')
nan_freq = (100 * (counts[3]/sum(counts))).round(4)
print(f'nan: {nan_freq}%')

0.0: 3389619
1.0: 13987736
2.0: 58540658
nan: 237347
nan: 0.3117%


Currently not sure if nan values are due to faulty encoding, and were supposed to be labeled as 3.0 or if that many exist.

In [ ]:
sample_data = dArray_sample.chunk({'variant':25600, 'sample':31})

In [ ]:
#SNP features picked from chromosomes 10, 4, 19, 1, and 5 based on Nature Journal
chromosomes_to_keep = ['10', '4', '19', '1', '5']
mask = np.isin(sample_data.variant.chrom, chromosomes_to_keep)
selected = sample_data.sel(variant=mask)

In [ ]:
selected

<xarray.DataArray 'genotype' (variant: 649512, sample: 32)>
dask.array<getitem, shape=(649512, 32), dtype=float32, chunksize=(25600, 31), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object '007_S_2058' '007_S_4620' ... '137_S_4672'
  * variant  (variant) <U14 'variant7238' 'variant7239' ... 'variant2199484'
    fid      (sample) object dask.array<chunksize=(31,), meta=np.ndarray>
    iid      (sample) object dask.array<chunksize=(31,), meta=np.ndarray>
    gender   (sample) object dask.array<chunksize=(31,), meta=np.ndarray>
    trait    (sample) int64 dask.array<chunksize=(31,), meta=np.ndarray>
    chrom    (variant) object dask.array<chunksize=(18362,), meta=np.ndarray>
    snp      (variant) object dask.array<chunksize=(18362,), meta=np.ndarray>
    cm       (variant) float64 dask.array<chunksize=(18362,), meta=np.ndarray>
    pos      (variant) int32 dask.array<chunksize=(18362,), meta=np.ndarray>
    a0       (variant) object dask.array<chunksize=(18362,), meta=np.ndarray>
    a1       (variant) object dask.array<chunksize=(18362,), meta=np.ndarray>

In [ ]:
df_sample = selected.to_dataframe()

In [ ]:
t = df_sample.reset_index('variant')


In [ ]:
df_sample

fid         iid gender  trait chrom          snp  \
variant        sample                                                         
variant7238    007_S_2058   82  007_S_2058      2      2     1    rs4477212   
               007_S_4620  559  007_S_4620      1      1     1    rs4477212   
               011_S_0002   17  011_S_0002      1      1     1    rs4477212   
               011_S_4547  260  011_S_4547      1      2     1    rs4477212   
               014_S_4668  434  014_S_4668      1      3     1    rs4477212   
...                        ...         ...    ...    ...   ...          ...   
variant2199484 129_S_0778   60  129_S_0778      1      3    19  kgp21396021   
               130_S_4250  503  130_S_4250      1      3    19  kgp21396021   
               136_S_0107  592  136_S_0107      2      2    19  kgp21396021   
               136_S_4269  568  136_S_4269      2      1    19  kgp21396021   
               137_S_4672   39  137_S_4672      1      3    19  kgp21396021   

                                cm       pos a0 a1  genotype  
variant        sample                                         
variant7238    007_S_2058   0.0000     82154  0  A       2.0  
               007_S_4620   0.0000     82154  0  A       2.0  
               011_S_0002   0.0000     82154  0  A       2.0  
               011_S_4547   0.0000     82154  0  A       2.0  
               014_S_4668   0.0000     82154  0  A       2.0  
...                            ...       ... .. ..       ...  
variant2199484 129_S_0778  98.9115  59097752  T  C       2.0  
               130_S_4250  98.9115  59097752  T  C       2.0  
               136_S_0107  98.9115  59097752  T  C       2.0  
               136_S_4269  98.9115  59097752  T  C       2.0  
               137_S_4672  98.9115  59097752  T  C       2.0  

[20784384 rows x 11 columns]